In [1]:
import warnings
warnings.filterwarnings('ignore')

import pickle

import sys
sys.path.append('/home/rupali/Documents/Master Thesis/jenga')

In [2]:
import random
import numpy as np
import pandas as pd

from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from jenga.basis import Dataset
from jenga.corruptions.generic import MissingValues, SwappedValues, CategoricalShift
from jenga.corruptions.numerical import Scaling, GaussianNoise
from jenga.cleaning.ppp import PipelinePerformancePrediction
from jenga.cleaning.outlier_detection import NoOutlierDetection, PyODKNNOutlierDetection, PyODIsolationForestOutlierDetection, PyODPCAOutlierDetection, PyODCBLOFOutlierDetection, PyODSOSOutlierDetection, SklearnOutlierDetection
from jenga.cleaning.imputation import MeanModeImputation, SklearnImputation, DatawigImputation
from jenga.cleaning.clean import Clean

import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [20, 10]

In [3]:
## use categorical columns as strings
def cat_cols_to_str(df):
    for col in df.columns:
        if pd.api.types.is_categorical_dtype(df[col]):
            df[col] = df[col].astype(str)

    return df

/home/rupali/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
def run_experiment(dataset_name, learner, param_grid, corruptions, fraction, cleaners, num_repetitions, categorical_precision_threshold=0.85, numerical_std_error_threshold=0.9):
    
    ## dataset
    dataset = Dataset(dataset_name)
    
    all_data = dataset.all_data
    attribute_names = dataset.attribute_names
    attribute_types = dataset.attribute_types
    
    ## categorical and numerical features
    categorical_columns = dataset.categorical_columns
    numerical_columns = dataset.numerical_columns
    print(f"Found {len(categorical_columns)} categorical and {len(numerical_columns)} numeric features \n")
    
    ## train and test data
    df_train, lab_train, df_test, lab_test = dataset.get_train_test_data()
    ### if we don't convert the categorical columns to str, the swapping corruption doesn't let us assign new values to the column: "Cannot setitem on a Categorical with a new category, set the categories first"
    df_train = cat_cols_to_str(df_train)
    df_test = cat_cols_to_str(df_test)
    
    ## pipeline performance prediction (ppp)
    ppp = PipelinePerformancePrediction(df_train, lab_train, df_test, lab_test, categorical_columns, numerical_columns, learner, param_grid)
    ppp_model = ppp.fit_ppp(df_train)
    ## the class column is added to the training data for Autogluon model fit
    if learner == None:
        df_train = df_train.loc[:, df_train.columns != 'class']
    
    ## generate corrupted data
    for _ in range(num_repetitions):
        df_corrupted, perturbations, cols_perturbed, summary_col_corrupt = ppp.get_corrupted(df_test, corruptions, fraction, num_repetitions)
    
    ## cleaning
    clean = Clean(df_train, df_corrupted, categorical_columns, numerical_columns, categorical_precision_threshold, numerical_std_error_threshold, ppp, ppp_model, cleaners)
    df_outliers, df_cleaned, corrupted_score_ppp, best_cleaning_score, cleaner_scores_ppp, summary_cleaners = clean(df_train, df_test, df_corrupted, cols_perturbed)
    
    ## results
    result = {
        'ppp_score_model': ppp.predict_score_ppp(ppp_model, df_test),
        'ppp_score_corrupted': corrupted_score_ppp,
        'ppp_score_cleaned': best_cleaning_score,
        'ppp_scores_cleaners': cleaner_scores_ppp
    }
    
    ## summary
    summary = {
        'dataset': dataset_name,
        'model': learner,
        'corruptions': summary_col_corrupt,
        'cleaners': summary_cleaners,
        'result': result
    }
    
    return summary #summary_col_corrupt, result

In [5]:
def save_obj(obj, name):
    with open('/home/rupali/Documents/Master Thesis/jenga/results/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open('/home/rupali/Documents/Master Thesis/jenga/results/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [6]:
datasets = [
    'thoracic_surgery',
    'cleve',
    'acute-inflammations',
    'hill-valley'
]

In [7]:
## model parameters
## models is a dict where key = leaner & value = param_grid
models = {SGDClassifier(loss='log'): {'learner__max_iter': [500, 1000, 5000], 
                                         'learner__penalty': ['l2', 'l1', 'elasticnet'], 
                                         'learner__alpha': [0.0001, 0.001, 0.01, 0.1]
                                        },
          RandomForestClassifier():{'learner__n_estimators': [100, 200, 500], 
                                    'learner__max_depth': [5, 10, 15]
                                   },
          None:None
         }

## make dict of multiple leraners and corresponding param_grids

In [8]:
fractions = [0.15, 0.25, 0.5, 0.75, 0.9]

In [9]:
cleaners = []
for od in [NoOutlierDetection, PyODKNNOutlierDetection, PyODIsolationForestOutlierDetection, PyODPCAOutlierDetection, PyODCBLOFOutlierDetection, PyODSOSOutlierDetection, SklearnOutlierDetection]:
    for imp in [MeanModeImputation, SklearnImputation, DatawigImputation]:
        cleaners.append((od, imp))

## Gaussian

In [10]:
corruptions = [[GaussianNoise]]

In [11]:
ind_results = {}

for fraction in fractions:
    ind_results[fraction] = []
    for learner, param_grid in models.items():
        for corruption in corruptions:
            for dataset in datasets:
                try:
                    ind_results[fraction].append(run_experiment(dataset, learner, param_grid, corruption, fraction, cleaners, 5))
                except ConnectionError:
                    print(f'Connection refused for dataset: {dataset}')
                    continue
                except ValueError:
                    print("Something went wrong with a value :(")
                    continue

Starting [get] request for the URL https://www.openml.org/api/v1/xml/data/list/limit/10000/offset/0/status/active/data_name/thoracic_surgery
2021-04-19 01:06:11,931 [INFO]  Starting [get] request for the URL https://www.openml.org/api/v1/xml/data/list/limit/10000/offset/0/status/active/data_name/thoracic_surgery
0.2469578s taken for [get] request for the URL https://www.openml.org/api/v1/xml/data/list/limit/10000/offset/0/status/active/data_name/thoracic_surgery
2021-04-19 01:06:12,179 [INFO]  0.2469578s taken for [get] request for the URL https://www.openml.org/api/v1/xml/data/list/limit/10000/offset/0/status/active/data_name/thoracic_surgery
pickle load data thoracic_surgery
2021-04-19 01:06:12,195 [INFO]  pickle load data thoracic_surgery
NumExpr defaulting to 4 threads.
2021-04-19 01:06:12,216 [INFO]  NumExpr defaulting to 4 threads.


Dataset: thoracic_surgery
Found 13 categorical and 3 numeric features 

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  44 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    3.4s finished
/home/rupali/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/rupali/.local/lib/python3.8/site-packages/sklearn/model_selection/_split.py:665: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(("The least populated class in y has only %d"



Generating corrupted training data on 94 rows... 

	perturbation: GaussianNoise: {'column': 'PRE5', 'fraction': 0.15, 'sampling': 'MAR'}

Generating corrupted training data on 94 rows... 

	perturbation: GaussianNoise: {'column': 'PRE5', 'fraction': 0.15, 'sampling': 'MNAR'}

Generating corrupted training data on 94 rows... 

	perturbation: GaussianNoise: {'column': 'PRE5', 'fraction': 0.15, 'sampling': 'MAR'}

Generating corrupted training data on 94 rows... 

	perturbation: GaussianNoise: {'column': 'AGE', 'fraction': 0.15, 'sampling': 'MCAR'}

Generating corrupted training data on 94 rows... 

	perturbation: GaussianNoise: {'column': 'PRE4', 'fraction': 0.15, 'sampling': 'MCAR'}

Applying cleaners... 

PPP score no cleaning: {'roc_auc_score': 0.5017543859649123, 'classification_report': {'F': {'precision': 0.797752808988764, 'recall': 0.9466666666666667, 'f1-score': 0.8658536585365854, 'support': 75}, 'T': {'precision': 0.2, 'recall': 0.05263157894736842, 'f1-score': 0.083333333333

/home/rupali/.local/lib/python3.8/site-packages/sklearn/model_selection/_split.py:665: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(("The least populated class in y has only %d"


...

/home/rupali/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Cleaner: (PyODIsolationForestOutlierDetection, DatawigImputation): {'roc_auc_score': 0.6294070512820512, 'classification_report': {'F': {'precision': 0.8297872340425532, 'recall': 1.0, 'f1-score': 0.9069767441860465, 'support': 78}, 'T': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 16}, 'accuracy': 0.8297872340425532, 'macro avg': {'precision': 0.4148936170212766, 'recall': 0.5, 'f1-score': 0.45348837209302323, 'support': 94}, 'weighted avg': {'precision': 0.688546853779991, 'recall': 0.8297872340425532, 'f1-score': 0.7525977238990599, 'support': 94}}}

Outlier detection method: PyODPCAOutlierDetection, Outlier Detection Score: {'Precision': 0.6575757575757576, 'Recall': 0.8017857142857143, 'F1-score': 0.6452830188679246, 'Accuracy': 0.7127659574468085}
Imputation method: MeanModeImputation, Imputation Score: {'Precision': nan, 'Recall': nan, 'F1-score': nan, 'Accuracy': nan, 'Mean Squared Error': 0.409574559420426}
Cleaner: (PyODPCAOutlierDetection, MeanModeImputation

/home/rupali/.local/lib/python3.8/site-packages/sklearn/model_selection/_split.py:665: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(("The least populated class in y has only %d"


Classifier for col: DGN reached 0.7266666666666666
Classifier for col: PRE6 reached 0.8733333333333333
Classifier for col: PRE7 reached 0.95
Classifier for col: PRE8 reached 0.85
Classifier for col: PRE9 reached 0.95
Classifier for col: PRE10 reached 0.8666666666666667
Classifier for col: PRE11 reached 0.8500000000000001


KeyboardInterrupt: 

In [ ]:
save_obj(ind_results, "gaussian/ind_results")

## Scaling

In [ ]:
corruptions = [[Scaling]]

In [ ]:
ind_results = {}

for fraction in fractions:
    ind_results[fraction] = []
    for learner, param_grid in models.items():
        for corruption in corruptions:
            for dataset in datasets:
                try:
                    ind_results[fraction].append(run_experiment(dataset, learner, param_grid, corruption, fraction, cleaners, 5))
                except ConnectionError:
                    print(f'Connection refused for dataset: {dataset}')
                    continue
                except ValueError:
                    print("Something went wrong with a value :(")
                    continue

In [ ]:
save_obj(ind_results, "scaling/ind_results")

## Missing

In [ ]:
corruptions = [[MissingValues]]

In [ ]:
ind_results = {}

for fraction in fractions:
    ind_results[fraction] = []
    for learner, param_grid in models.items():
        for corruption in corruptions:
            for dataset in datasets:
                try:
                    ind_results[fraction].append(run_experiment(dataset, learner, param_grid, corruption, fraction, cleaners, 5))
                except ConnectionError:
                    print(f'Connection refused for dataset: {dataset}')
                    continue
                except ValueError:
                    print("Something went wrong with a value :(")
                    continue

In [ ]:
save_obj(ind_results, "missing/ind_results")

## Categorical Shift

In [ ]:
corruptions = [[CategoricalShift]]

In [ ]:
ind_results = {}

for fraction in fractions:
    ind_results[fraction] = []
    for learner, param_grid in models.items():
        for corruption in corruptions:
            for dataset in datasets:
                try:
                    ind_results[fraction].append(run_experiment(dataset, learner, param_grid, corruption, fraction, cleaners, 5))
                except ConnectionError:
                    print(f'Connection refused for dataset: {dataset}')
                    continue
                except ValueError:
                    print("Something went wrong with a value :(")
                    continue

In [ ]:
save_obj(ind_results, "categorical_shift/ind_results")